In [1]:
import sys
sys.path.insert(0,'../../backend')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import gym
import stable_baselines
from stable_baselines import A2C, ACER, ACKTR, DQN, DDPG, SAC, PPO1, PPO2, TD3, TRPO
from stable_baselines.common.env_checker import check_env
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.gail import ExpertDataset, generate_expert_traj

import webotsgym as wg

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
from webotsgym.utils import exponential_decay
from webotsgym.env.reward import step_pen_exp

class MyEval(wg.WbtReward):
    def __init__(self, env, config):
        super(MyEval, self).__init__(env, config)

    def calc_reward(self):
        target_distance = self.env.get_target_distance(False)
        if target_distance < 0.1:
            return 10000 + 5000 * (1 - abs(self.env.state.speed))
        else:
            reward = 0
            reward += -2
            
            if len(self.env.distances) > 0:
                diff = self.env.distances[-1] - self.env.get_target_distance(False)
                #print("Diff: ", diff)
                
                speed_max = 0.03
                if abs(self.env.state.speed) > speed_max :
                    speed_max = abs(self.env.state.speed)
                    print("+++++++++++++++++ speed: ", speed_max)
                    
                diff_max = speed_max * 1000 / (32 * config.sim_step_every_x)
                
                if diff > diff_max:
                    print("----------------- diff: ", diff, diff_max, " > diff_max")
                # print("Speed: ", self.env.state.speed)

                diff_rew = 0.5 * ( diff  / diff_max) / abs(self.env.state.speed / speed_max)
                #if abs(diff_rew) > 3:
                   # print("========================")
                   # print("Diff_rew:" , diff_rew)
                   # print("Diff%: ", diff  / diff_max)
                   # print("Speed%: ", abs(self.env.state.speed / speed_max))
                   # print("========================")
                    
                if diff_rew > 3:
                    reward += 3
                elif diff_rew < -3:
                    reward -= 3
                else:
                    reward += diff_rew
                    
            if self.env.state.action_denied:
                reward += -5
        
            if self.env.state.touching:
                reward += -100
        return reward

    def check_done(self):
        if self.env.total_reward < -5000:
            print("reward boundary, reward: ", self.env.total_reward)
            return True
        if self.env.get_target_distance(False) < 0.1:
            print("target reached, reward: ", self.env.total_reward)
            return True
        if self.env.total_reward > 25000:
            print("Stop hacking!, reward: ", self.env.total_reward)
            return True
        return False

In [3]:
config = wg.WbtConfig()
config.world_size = 3
config.num_obstacles = 0
config.sim_mode = wg.config.SimSpeedMode.FAST
config.sim_step_every_x = 10
config.relative_action = True
config.direction_type = wg.config.DirectionType.STEERING

In [4]:
env = wg.WbtGym(train=True, 
                evaluate_class=MyEval,
#                 action_class = wg.WbtActDiscrete(config, dspeed=0.05, ddir=0.05),
                config=config)

../../backend/webotsgym/env/webotenv.py:111: UserWarning: Relative property of action class is overwritten by config.relative_action.
  warnings.warn("Relative property of action class is overwritten "


Accepting on Port:  10201


In [5]:
model_name = "3x3_reach_target_v5"
model = PPO1("MlpPolicy", env, timesteps_per_actorbatch = 5000, tensorboard_log="./{}".format(model_name))





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.









Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
while True:
    model.learn(total_timesteps=500000)
    model.save("./{}".format(model_name))


reward boundary, reward:  -5102.15402090258
----------------- diff:  0.09587537993207418 0.09375  > diff_max
reward boundary, reward:  -5008.376935909232
----------------- diff:  0.09583493180441083 0.09375  > diff_max
reward boundary, reward:  -5098.226146618448
target reached, reward:  -953.7493465240144
----------------- diff:  0.09533753398455086 0.09375  > diff_max
reward boundary, reward:  -5011.2022842597
----------------- diff:  0.09521677110368665 0.09375  > diff_max
reward boundary, reward:  -5033.1094885022985
----------------- diff:  0.10926217152282536 0.09375  > diff_max
reward boundary, reward:  -5037.957966016836
----------------- diff:  0.09901757371178477 0.09375  > diff_max
reward boundary, reward:  -5092.074545729583
reward boundary, reward:  -5042.757786364394
reward boundary, reward:  -5039.530695939796
reward boundary, reward:  -5029.096572910092
reward boundary, reward:  -5036.3197517205135
reward boundary, reward:  -5045.546452101389
target reached, reward:  -

target reached, reward:  -3374.873377159996
reward boundary, reward:  -5002.999305988625
----------------- diff:  0.1091416410078283 0.09375  > diff_max
reward boundary, reward:  -5046.626165250505
target reached, reward:  -74.6200755394706
reward boundary, reward:  -5050.215051321832
reward boundary, reward:  -5001.890387323403
reward boundary, reward:  -5001.4723881713335
reward boundary, reward:  -5050.670979021563
----------------- diff:  0.09648296477675089 0.09375  > diff_max
----------------- diff:  0.09390947941528385 0.09375  > diff_max
reward boundary, reward:  -5081.397549225317
----------------- diff:  0.09495532171818441 0.09375  > diff_max
----------------- diff:  0.09465535250516054 0.09375  > diff_max
target reached, reward:  -3939.8165717409393
reward boundary, reward:  -5009.529402398625
reward boundary, reward:  -5016.22026304573
reward boundary, reward:  -5091.326703639975
reward boundary, reward:  -5002.392176542607
----------------- diff:  0.09790443649775149 0.09

reward boundary, reward:  -5074.514024860041
reward boundary, reward:  -5051.329974612096
reward boundary, reward:  -5096.301315035938
----------------- diff:  0.09874687641237373 0.09375  > diff_max
reward boundary, reward:  -5063.18840473721
target reached, reward:  -1661.302800296745
----------------- diff:  0.0999436771750013 0.09375  > diff_max
reward boundary, reward:  -5026.725158573791
target reached, reward:  -456.53438631260605
target reached, reward:  -283.6631790865766
reward boundary, reward:  -5005.596048438363
----------------- diff:  0.11190233638182234 0.09375  > diff_max
reward boundary, reward:  -5060.272149054964
target reached, reward:  -1223.4956065437182
reward boundary, reward:  -5032.596172838401
----------------- diff:  0.09553099542553056 0.09375  > diff_max
target reached, reward:  -2969.8767076805966
----------------- diff:  0.09379539538620141 0.09375  > diff_max
reward boundary, reward:  -5003.534593555845
reward boundary, reward:  -5012.878742335046
targ